##### Code Etabli

In [1]:
import os
import pandas as pd
import numpy as np
import math

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import gc


In [2]:
# Calcule la quantité de Mg à utiliser
def calcul_quantite_mg(P,S,t,e,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * e) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

# Calcule de la longueur du fil fourré et de la masse de fonte limite dans le four de maintien
def calcul_longueur_fil_et_limite_four(
    masse_grappe, nb_moules_heure, masse_fonte_poche, masse_fonte_coulee, pct_mg_fonte_coulee,
    pct_Soufre, tempera_fonte_poche,
    pct_rendement_mg, masse_fil, masse_mg_fil, masse_fonte_coulee_min, masse_fonte_coulee_max, pct_mg_fonte_coulee_min,
    pct_mg_fonte_coulee_max, pct_perdu_mg_coulee_min, pct_perdu_mg_poche_min, temps_traitement, temps_gs,
    heure_lancement):

    # Temps en minute admissible avant l'ajout dans le four de maintien
    # avant d'atteindre la fonte minimal (en Kg) dans le four de maintien
    consommation_fonte_min = nb_moules_heure * masse_grappe / 60  # en kg/min
    fonte_four_consommable = masse_fonte_coulee - masse_fonte_coulee_min
    temps_epuis_fonte = fonte_four_consommable/consommation_fonte_min
    delai_avt_traitement_fonte_four = temps_epuis_fonte - temps_traitement



    # Temps en minute admissible avant l'ajout dans le four de maintien
    # avant d'atteindre le pourcentage minimal (%) dans le four de maintien
    pct_mg_four_consommable = pct_mg_fonte_coulee - pct_mg_fonte_coulee_min
    temps_epuis_mg = pct_mg_four_consommable/pct_perdu_mg_coulee_min
    delai_avt_traitement_mg_four = temps_epuis_mg - temps_traitement


    # Temps en minute avant de lancer le traitement (Du four de fusion au four de maintien)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)

    # print(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    # mise a jour de la masse fonte avant ajout poche mais après consomation de la fonte
    masse_fonte_four_consommer = (temps_traitement + delai_avt_traitement)*consommation_fonte_min 
    masse_four_limite =  masse_fonte_coulee - masse_fonte_four_consommer


    # mise a jour de la masse mg avant ajout poche après consomation de mg
    pct_mg_four_consommer = (temps_traitement + delai_avt_traitement)*pct_perdu_mg_coulee_min
    pct_mg_four_limite = pct_mg_fonte_coulee - pct_mg_four_consommer


    # % de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    K = ( pct_mg_fonte_coulee_max*(masse_four_limite + masse_fonte_poche) - pct_mg_four_limite*masse_four_limite )/ masse_fonte_poche

    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    pct_mg_fil = masse_mg_fil/masse_fil *100
    Q = calcul_quantite_mg(masse_fonte_poche,pct_Soufre,temps_gs,pct_perdu_mg_poche_min,tempera_fonte_poche,pct_rendement_mg,pct_mg_fil,K) # en Kg


    # Longueur de fil pour avoir la masse de Mg manquante
    longueur_fil_necessaire = Q / (masse_fil* 1e-3)   # en m

    return K,delai_avt_traitement,pct_mg_four_limite,masse_four_limite,longueur_fil_necessaire

def Calcule_Temps_prochain_traitement(delai_avt_traitement,heure_lancement):
    # Trouver les positions des unités de temps dans la chaîne
    pos_hr = heure_lancement.find('hr')
    pos_min = heure_lancement.find('min')
    pos_sec = heure_lancement.find('s')

    # Extraire les valeurs numériques
    now_hours = int(heure_lancement[:pos_hr].strip()) 
    now_minutes = int(heure_lancement[pos_hr+3:pos_min].strip()) 
    now_seconds = int(heure_lancement[pos_min+4:pos_sec].strip()) 


    # Calculer le temps total en secondes
    total_seconds = now_hours * 3600 + now_minutes * 60 + now_seconds + delai_avt_traitement*60

    # Calculer les nouvelles heures, minutes et secondes
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)

    heure_prochain_lancement = f"{hours} hr {minutes} min {seconds} s"
    return heure_prochain_lancement

def export_result(df, dossier_data):
    """
    """
    # Créer le chemin complet du nouveau fichier Excel
    fichier_resultats = os.path.join(dossier_data, 'Resultats.xlsx')

    workbook = Workbook()
    feuille = workbook.active 

    # Écrire le DataFrame dans la feuille
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            feuille.cell(row=r_idx, column=c_idx, value=value)

    # Sauvegarder le classeur
    workbook.save(fichier_resultats)
    workbook.close()
    gc.collect()
    return 

def main_fct(chemin_fichier, dossier_courant):
    # Lecture de la première feuille du fichier Excel
    df = pd.read_excel(chemin_fichier, engine='openpyxl')

    # Supprimer les lignes vides
    df2 = df.dropna(how='all').reset_index(drop=True)

    # Extraction des paramètres Généraux de traitement GS
    pct_rendement_mg, masse_fil, masse_mg_fil, masse_fonte_coulee_min, masse_fonte_coulee_max, pct_mg_fonte_coulee_min= [
        pd.to_numeric(df2.iloc[1, i], errors='coerce') for i in range(0, 6)
    ]

    pct_mg_fonte_coulee_max, pct_perdu_mg_coulee_min, pct_perdu_mg_poche_min, temps_traitement, temps_gs = [
        pd.to_numeric(df2.iloc[1, i], errors='coerce') for i in range(6, 11)
    ]

    heure_lancement = df2.iloc[1, 11]

    # Extraction des variables du Fours de fusion
    pct_Soufre, tempera_fonte_poche = [
        pd.to_numeric(df2.iloc[4, i], errors='coerce') for i in range(5, 7)
    ]

    # Extraction des variables du Fours de couléee
    masse_grappe, nb_moules_heure, masse_fonte_poche, masse_fonte_coulee, pct_mg_fonte_coulee = [
        pd.to_numeric(df2.iloc[7, i], errors='coerce') for i in range(0, 5)
    ]


    # Calcul de la longueur de fil nécessaire
    K,delai_avt_traitement,pct_mg_fonte_coulee_limite,masse_fonte_coulee_limite,longueur_fil_necessaire = calcul_longueur_fil_et_limite_four(
            masse_grappe, nb_moules_heure, masse_fonte_poche, masse_fonte_coulee, pct_mg_fonte_coulee,
        pct_Soufre, tempera_fonte_poche,
        pct_rendement_mg, masse_fil, masse_mg_fil, masse_fonte_coulee_min, masse_fonte_coulee_max, pct_mg_fonte_coulee_min,
        pct_mg_fonte_coulee_max, pct_perdu_mg_coulee_min, pct_perdu_mg_poche_min, temps_traitement, temps_gs,
        heure_lancement)
        
    # Liste des noms de valeurs à rechercher
    output_name = [
        'Pourcentage de magnésium dans le four de coulée au moment de l\'ajout de la poche (%)',
        'Masse de la fonte dans le four de coulée au moment de l\'ajout de la poche (en Kg)',
        'Pourcentage de magnésium dans le four de coulée après  l\'ajout de la poche (%)',
        'Masse de la fonte dans le four de coulée  après l\'ajout de la poche (en Kg)',
        'Temps restant avant le lancement du prochain traitement (en seconde)',
        'Heure de lancement du prochain traitement',
        'Longueur théorique du fil fourré à utiliser (en m)'

    ]

    # Listes pour stocker les résultats
    indices_lignes_output = []
    noms_colonnes_output = []

    # Parcours du DataFrame pour trouver chaque valeur spécifique
    for target_value in output_name:
        found = False
        for index, row in df.iterrows():
            for col_name in df.columns:
                if row[col_name] == target_value:
                    indices_lignes_output.append(index)
                    noms_colonnes_output.append(col_name)
                    found = True
                    break
            if found:
                break


    heure_prochain_lancement = Calcule_Temps_prochain_traitement(delai_avt_traitement,heure_lancement)
    df_res = df.copy()

    masse_mg_coulee = pct_mg_fonte_coulee_limite* masse_fonte_coulee_limite/100  
    masse_mg_poche = K*masse_fonte_poche/100  
    pct_mg_coulee_apres_ajout = (masse_mg_coulee+masse_mg_poche)/(masse_fonte_coulee_limite+masse_fonte_poche)*100
    # df_res.columns.values[10:12] = res_name
    df_res.loc[indices_lignes_output[0] +1, noms_colonnes_output[0]] = pct_mg_fonte_coulee_limite
    df_res.loc[indices_lignes_output[1] +1, noms_colonnes_output[1]] = masse_fonte_coulee_limite
    df_res.loc[indices_lignes_output[2]+1, noms_colonnes_output[2]] = pct_mg_coulee_apres_ajout
    df_res.loc[indices_lignes_output[3]+1, noms_colonnes_output[3]] = masse_fonte_coulee_limite+masse_fonte_poche
    df_res.loc[indices_lignes_output[4]+1, noms_colonnes_output[4]] = delai_avt_traitement
    df_res.loc[indices_lignes_output[5]+1, noms_colonnes_output[5]] = heure_prochain_lancement
    df_res.loc[indices_lignes_output[6]+1, noms_colonnes_output[6]] = longueur_fil_necessaire


    
    export_result(df_res, dossier_courant )
    return 


In [3]:

if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'Sphérodisation.xlsm')
    # On recupere le chemin du dossier data
    dossier_courant = os.path.dirname(chemin_fichier)
    # Solve problème
    main_fct(chemin_fichier, dossier_courant)

##### Variables Globales 

In [4]:
#### Variables Globales 


# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg
Poche = False 

temps_serie = 5  # en min
temps_traitement = 10 # en min
temps_serie_courant = temps_serie
temps_gs = 4 # en min



# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg


##### Fonction Globales 

In [5]:
# Pour calculer la longueur du fil fourré 
def calcul_quantite_mg(P,S,t,e,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * e) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

def calcul_longueur_fil( PPT, S, TPT,
    R, masse_fil, masse_mg_fil, Mgmax, eP, 
    PFClimite, Mglimite ):

    # % de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    K = ( Mgmax*(PFClimite + PPT) - Mglimite*PFClimite )/ PPT

    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg maximal
    Mgfil = masse_mg_fil/masse_fil *100
    Q = calcul_quantite_mg(PPT,S,temps_gs,eP,TPT,R,Mgfil,K) # en Kg


    # Longueur de fil pour avoir la masse de Mg manquante
    L = Q / (masse_fil* 1e-3)   # en m

    return K, L




# Pour le passage d'un etat à l'autre (consom, Serie, Panne, Fin)
def PendantChangement_serie ( Mg, PFC) :
    # Variables dans ce programme
    global temps_serie, temps_serie_courant

    # pendant Temps_Serie :
    Mg -= eC # perte de Mg par minute ou par seconde
    temps_serie_courant -= 1 # pas de temps en minute
    
    etat_suivant = "etat_Serie"
    if temps_serie_courant == 0:
        temps_serie_courant = temps_serie  # Réinitialiser le temps de série
        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant

def PendantConsommation ( Mg, PFC):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire


    # On produit les moules !! 
    masse_grappe_i = liste_masses_grappes_moules[0]
    cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
    quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
    
    # La cadence de consommation en kg/min du i-ème modèle
    cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
    
    # Quantités de moules du i-ème modèle produits en une minute
    cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min

    quantite_moules_restants = quantite_moules_a_produire_i - cadence_moule_i


    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC


    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat_suivant = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat_suivant = "etat_Fin"

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat_suivant= "etat_Consom"
    
    return Mg, PFC, etat_suivant

def PendantPanne ( Mg, PFC) :
    # Variables dans ce programme
    global Panne

    # pendant Temps_Panne non définis :
    Mg -= eC # perte de Mg par minute ou par seconde


    etat_suivant = "etat_Panne"
    if not Panne :
        etat_suivant = "etat_Consom"
    
    return Mg, PFC, etat_suivant

def gerer_etat(Mg, PFC, etat_courant):
    """
    Gère les transitions entre différents états et effectue les opérations associées.

    Parameters:
        etat_courant (str): L'état actuel du système ("etat_Consom", "etat_Serie", "etat_Fin").
        Mg (float): La quantité de matériau Mg.
        PFC (float): La quantité de PFC disponible.

    Returns:
        Tuple: Retourne les variables mises à jour pour Mg, PFC, etat_suivant.
    """

    # Constantes dans ce programme
    global running


    # Gérer l'état de consommation
    if etat_courant == "etat_Consom":
        Mg, PFC, etat_suivant = PendantConsommation ( Mg, PFC)

    # Gérer l'état de série
    elif etat_courant == "etat_Serie":
        Mg, PFC, etat_suivant = PendantChangement_serie ( Mg, PFC)

    # Gérer l'état de panne
    elif etat_courant == "etat_Panne":
        Mg, PFC, etat_suivant = PendantPanne ( Mg, PFC)

    # Gérer l'état de fin
    elif etat_courant == "etat_Fin":
        running = False  # Fin du processus
        etat_suivant = "etat_Fin"
    return Mg, PFC, etat_suivant




def gerer_poche(Mg, PFC):
    """
    Gère l'ajout de la poche. Si Poche est True, met à jour Mg et PFC.
    
    Args:
    Mg (float): La quantité actuelle de Mg.
    PFC (float): La quantité actuelle de Fonte.

    
    Returns:
    Mg (float): La nouvelle quantité de Mg.
    PFC (float): La nouvelle quantité de fonte.
    """

    # Constantes dans ce programme
    global Mgmax, poche_i
    # Variables dans ce programme
    global Poche

    if Poche:
        Mg = Mgmax
        PFC += poche_i
        Poche = False
    return Mg, PFC


In [6]:
# Pour calculer le temps du prochain Traitement GS
def Calcule_Temps_prochain_traitement(delai_avt_traitement,heure_lancement):
    # Trouver les positions des unités de temps dans la chaîne
    pos_hr = heure_lancement.find('hr')
    pos_min = heure_lancement.find('min')
    pos_sec = heure_lancement.find('s')

    # Extraire les valeurs numériques
    now_hours = int(heure_lancement[:pos_hr].strip()) 
    now_minutes = int(heure_lancement[pos_hr+3:pos_min].strip()) 
    now_seconds = int(heure_lancement[pos_min+4:pos_sec].strip()) 


    if delai_avt_traitement < 0 :
        print("Lancer le Traitement GS !")

    # Calculer le temps total en secondes
    total_seconds = now_hours * 3600 + now_minutes * 60 + now_seconds + delai_avt_traitement*60

    # Calculer les nouvelles heures, minutes et secondes
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)

    heure_prochain_lancement = f"{hours} hr {minutes} min {seconds} s"
    return heure_prochain_lancement    


# Calcule du temps limite avant lancement prochain traitement
def calculer_temps_epuis_PFC(PFC):
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire, temps_serie

    # Calcul de la fonte four consommable
    PFCconsommable = PFC - PFCmin

    # Initialisation du temps total
    temps_total = 0

    # Boucle sur chaque modèle de moules
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min
        
        # Fonte nécessaire pour produire tous les moules de ce modèle
        fonte_necessaire_i = quantite_moules_a_produire_i * masse_grappe_i

        if PFCconsommable >= fonte_necessaire_i:
            # Si la fonte est suffisante pour produire tous les moules
            temps_pour_produire_i = fonte_necessaire_i / cadence_fonte_i
            temps_total += temps_pour_produire_i
            PFCconsommable -= fonte_necessaire_i

            # Ajouter le temps de la série s'il y a d'autres modèles à produire
            if i < len(liste_masses_grappes_moules) - 1:
                temps_total += temps_serie
        else:
            # Si la fonte n'est pas suffisante, calcul du temps possible avec la fonte restante
            temps_possible = PFCconsommable / cadence_fonte_i
            temps_total += temps_possible
            break  # On arrête la boucle, car il n'y a plus de fonte

    return temps_total

def calcul_temps_limite(PFC, Mg):
    
    global Mgmin, eC, temps_traitement

    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre la fonte minimal (en Kg) dans le four de coulée
    temps_epuis_PFC = calculer_temps_epuis_PFC(PFC)
    delai_avt_traitement_fonte_four = temps_epuis_PFC - temps_traitement


    
    # Temps en minute admissible avant l'ajout dans le four de coulée
    # avant d'atteindre le pourcentage minimal (%) dans le four de coulée
    Mgconsommable = Mg - Mgmin
    temps_epuis_Mg = Mgconsommable/eC
    delai_avt_traitement_mg_four = temps_epuis_Mg - temps_traitement


    # Temps en minute avant de lancer le traitement (Du four de fusion au four de coulée)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    tempslimite = temps_traitement + delai_avt_traitement
    # mise a jour de la masse mg avant ajout poche après consomation de mg
    Mgconsommer = tempslimite*eC
    Mgconsommer = (temps_traitement + delai_avt_traitement)*eC
    Mglimite = Mg - Mgconsommer



    # Mise à jour de la masse de fonte avant ajout dans le four après consommation pendant
    # temps_traitement + delai_avt_traitement
    PFCconsommer = 0
    for i in range(len(liste_masses_grappes_moules)):
        masse_grappe_i = liste_masses_grappes_moules[i]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[i]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[i]

        # Calcul de la cadence de fonte en kg/min pour le modèle i
        cadence_fonte_i = (cadence_moule_par_heure_i / 60) * masse_grappe_i  # en kg/min

        # Vérification du temps restant par rapport au temps nécessaire pour produire tous les moules de ce modèle
        temps_pour_produire_i = quantite_moules_a_produire_i / (cadence_fonte_i / masse_grappe_i)
        
        if tempslimite > temps_pour_produire_i:
            # Si le délai est supérieur au temps nécessaire, consommer toute la fonte pour ce modèle
            PFCconsommer += cadence_fonte_i * temps_pour_produire_i
            tempslimite -= temps_pour_produire_i

            if i < len(liste_masses_grappes_moules) - 1:
                tempslimite -= temps_serie
        else:
            # Sinon, consommer la fonte en fonction du temps restant
            PFCconsommer += cadence_fonte_i * tempslimite
            break

    PFClimite =  PFC - PFCconsommer


    return delai_avt_traitement, Mglimite, PFClimite




# def gerer_Prevention(Mg, PFC, etat_courant):

#     delai_avt_traitement, Mglimite, PFClimite = calcul_temps_limite(PFC, Mg)
#     if delai_avt_traitement == 0 :

#     return Mg, PFC, etat_suivant



##### Fonctions interactives 

In [7]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import time

def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    global Poche, poche_i

    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t
    etat_courant = "etat_Consom"  
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")

    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC
    global Poche, poche_i, Mgmax
    
    while running:
        # Mettre à jour les données de temps
        timedata.append(t)
        
        # Appeler la fonction pour gérer l'état
        Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)

        # Si l'on ajoute de la poche
        Mg, PFC = gerer_poche(Mg, PFC)

        
        # Mettre à jour les données pour les graphiques
        Mgdata.append(Mg)
        PFCdata.append(PFC)

        with fig1.batch_update():
            scatter1.update(x=timedata, y=Mgdata)
        
        with fig2.batch_update():
            scatter2.update(x=timedata, y=PFCdata)
            
        t += time_step
        etat_courant = etat_suivant
        
        # Pause pour simuler le temps entre les mises à jour
        time.sleep(time_step)

def start_plot(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    running = True

    global etat_courant, Panne
    Panne = False
    etat_courant = "etat_Consom"


    # # Démarrer la mise à jour des données dans un nouveau thread ou processus
    import threading
    thread = threading.Thread(target=update_data)
    thread.start()


def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_courant, Mg, PFC
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_courant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    with fig1.batch_update():
        scatter1.update(x=timedata, y=Mgdata)
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)

def serie_plot(button):
    """Fonction pour passer à l'état de série."""
    global etat_courant
    etat_courant = "etat_Serie"


def panne_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Panne
    Panne = True
    etat_courant = "etat_Panne"

def poche_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Poche
    Poche = True
    etat_courant = "etat_Consom"


def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False


def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start")
    reset_button = widgets.Button(description="Reset")
    stop_button = widgets.Button(description="Stop")
    poche_button = widgets.Button(description="Poche")
    serie_button = widgets.Button(description="Série")
    panne_button = widgets.Button(description="Panne")
    poche_button = widgets.Button(description="Poche")



    start_button.on_click(start_plot)
    reset_button.on_click(reset_plot)
    stop_button.on_click(stop_plot)
    poche_button.on_click(poche_plot)
    serie_button.on_click(serie_plot)
    panne_button.on_click(panne_plot)


    display(widgets.HBox([start_button, reset_button, stop_button, poche_button, serie_button, panne_button]), fig1, fig2)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig1, scatter1, fig2, scatter2
    fig1, scatter1, fig2, scatter2 = create_figures()
    create_buttons()

# Appel de la fonction principale pour démarrer l'application
main()

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '3ec163e6-fadc-4ad8-ac96-04dbddb91ace'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '9c88146e-0708-4f54-9c0d-4e5ffa542672'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

###### Sans poche

In [9]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import time
from threading import Timer

def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")

    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    if running:
        # Mettre à jour les données de temps
        timedata.append(t)

        # Appeler la fonction pour gérer l'état
        Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)

        # Si l'on ajoute de la poche
        Mg, PFC = gerer_poche(Mg, PFC)

        # Mettre à jour les données pour les graphiques
        Mgdata.append(Mg)
        PFCdata.append(PFC)

        with fig1.batch_update():
            scatter1.update(x=timedata, y=Mgdata)
        
        with fig2.batch_update():
            scatter2.update(x=timedata, y=PFCdata)
        
        t += time_step
        etat_courant = etat_suivant

        # Programmer la prochaine mise à jour
        Timer(time_step, update_data).start()

def start_plot(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    running = True

    global etat_courant, Panne
    Panne = False
    etat_courant = "etat_Consom"

    # Démarrer la mise à jour des données
    update_data()

def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_courant, Mg, PFC
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_courant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    with fig1.batch_update():
        scatter1.update(x=timedata, y=Mgdata)
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)

def serie_plot(button):
    """Fonction pour passer à l'état de série."""
    global etat_courant
    etat_courant = "etat_Serie"

def panne_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Panne
    Panne = True
    etat_courant = "etat_Panne"

def poche_plot(button):
    """Fonction pour passer à l'état de poche."""
    global etat_courant, Poche
    Poche = True
    etat_courant = "etat_Consom"

def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start")
    reset_button = widgets.Button(description="Reset")
    stop_button = widgets.Button(description="Stop")
    poche_button = widgets.Button(description="Poche")
    serie_button = widgets.Button(description="Série")
    panne_button = widgets.Button(description="Panne")

    start_button.on_click(start_plot)
    reset_button.on_click(reset_plot)
    stop_button.on_click(stop_plot)
    poche_button.on_click(poche_plot)
    serie_button.on_click(serie_plot)
    panne_button.on_click(panne_plot)

    display(widgets.HBox([start_button, reset_button, stop_button, poche_button, serie_button, panne_button]), fig1, fig2)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig1, scatter1, fig2, scatter2
    fig1, scatter1, fig2, scatter2 = create_figures()
    create_buttons()

# Appel de la fonction principale pour démarrer l'application
main()


FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'cb987942-bb6e-45a8-8c6d-e4e0a5dbaa1d'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '2abcea84-ae49-4192-9e17-11a1b989d070'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

In [10]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import asyncio

def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()
    scatter1 = fig1.add_scatter(mode='lines+markers').data[0]
    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")

    fig2 = go.FigureWidget()
    scatter2 = fig2.add_scatter(mode='lines+markers').data[0]
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, scatter1, fig2, scatter2

async def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    while running:
        # Mettre à jour les données de temps
        timedata.append(t)

        # Appeler la fonction pour gérer l'état
        Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)

        # Si l'on ajoute de la poche
        Mg, PFC = gerer_poche(Mg, PFC)

        # Mettre à jour les données pour les graphiques
        Mgdata.append(Mg)
        PFCdata.append(PFC)

        with fig1.batch_update():
            scatter1.update(x=timedata, y=Mgdata)
        
        with fig2.batch_update():
            scatter2.update(x=timedata, y=PFCdata)
        
        t += time_step
        etat_courant = etat_suivant

        # Attendre avant la prochaine mise à jour
        await asyncio.sleep(time_step)

def start_plot(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    running = True

    global etat_courant, Panne
    Panne = False
    etat_courant = "etat_Consom"

    # Démarrer la mise à jour des données
    asyncio.ensure_future(update_data())

def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_courant, Mg, PFC
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_courant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    with fig1.batch_update():
        scatter1.update(x=timedata, y=Mgdata)
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)

def serie_plot(button):
    """Fonction pour passer à l'état de série."""
    global etat_courant
    etat_courant = "etat_Serie"

def panne_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Panne
    Panne = True
    etat_courant = "etat_Panne"

def poche_plot(button):
    """Fonction pour passer à l'état de poche."""
    global etat_courant, Poche
    Poche = True
    etat_courant = "etat_Consom"

def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start")
    reset_button = widgets.Button(description="Reset")
    stop_button = widgets.Button(description="Stop")
    poche_button = widgets.Button(description="Poche")
    serie_button = widgets.Button(description="Série")
    panne_button = widgets.Button(description="Panne")

    start_button.on_click(start_plot)
    reset_button.on_click(reset_plot)
    stop_button.on_click(stop_plot)
    poche_button.on_click(poche_plot)
    serie_button.on_click(serie_plot)
    panne_button.on_click(panne_plot)

    display(widgets.HBox([start_button, reset_button, stop_button, poche_button, serie_button, panne_button]), fig1, fig2)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig1, scatter1, fig2, scatter2
    fig1, scatter1, fig2, scatter2 = create_figures()
    create_buttons()

# Appel de la fonction principale pour démarrer l'application
main()


FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '2bf3da14-fbeb-49f6-87ea-5e4ee2a9a637'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '041626fa-cfdd-459c-8f93-cd5bae12e23b'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})

In [15]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import asyncio

def initialize_variables(initial_time_step, initial_t, Mgmin=0.05, Mgmax=0.1, Mglimite=0.02):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC

    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t
    etat_courant = "etat_Consom"
    Mg = 0.045  # Valeur initiale pour Mg
    PFC = 3500  # Valeur initiale pour PFC
    Mgmin = Mgmin  # Valeur minimale de Mg
    Mgmax = Mgmax  # Valeur maximale de Mg
    Mglimite = Mglimite  # Valeur limite de Mg

def create_figures():
    """Crée et configure les figures Plotly pour Mg et PFC."""
    fig1 = go.FigureWidget()
    fig2 = go.FigureWidget()
    
    # Initialisation des courbes vides
    fig1.add_scatter(mode='lines+markers')  # Courbe pour Mg
    fig2.add_scatter(mode='lines+markers')  # Courbe pour PFC

    fig1.update_layout(title="Consommation de Mg", xaxis_title="Temps", yaxis_title="Mg")
    fig2.update_layout(title="Consommation de Fonte", xaxis_title="Temps", yaxis_title="Fonte")

    return fig1, fig2

def update_curve_color(x, y):
    """Met à jour les couleurs des segments en fonction des valeurs de Mg."""
    segments = []
    colors = []
    start_idx = 0

    # Détermine les segments et leurs couleurs
    for i in range(1, len(y)):
        if y[i] >= Mgmax:
            color = 'green'
        elif y[i] >= Mgmin and y[i] < Mgmax:
            color = 'green'
        elif y[i] >= Mglimite and y[i] < Mgmin:
            color = 'orange'
        else:
            color = 'red'
        
        if color != colors[-1] if colors else None:
            segments.append((x[start_idx:i], y[start_idx:i]))
            colors.append(color)
            start_idx = i

    # Ajouter le dernier segment
    if start_idx < len(x):
        segments.append((x[start_idx:], y[start_idx:]))
    
    return segments, colors

async def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    global etat_courant, Mg, PFC, Mgmin, Mgmax, Mglimite

    while running:
        # Mettre à jour les données de temps
        timedata.append(t)

        # Appeler la fonction pour gérer l'état
        Mg, PFC, etat_suivant = gerer_etat(Mg, PFC, etat_courant)

        # Si l'on ajoute de la poche
        Mg, PFC = gerer_poche(Mg, PFC)

        # Mettre à jour les données pour les graphiques
        Mgdata.append(Mg)
        PFCdata.append(PFC)

        # Mise à jour des courbes avec les couleurs appropriées
        segments, colors = update_curve_color(timedata, Mgdata)

        with fig1.batch_update():
            fig1.data = []  # Effacer les données existantes
            for segment, color in zip(segments, colors):
                fig1.add_scatter(x=segment[0], y=segment[1], mode='lines', line=dict(color=color))

        with fig2.batch_update():
            scatter2.update(x=timedata, y=PFCdata)
        
        t += time_step
        etat_courant = etat_suivant

        # Attendre avant la prochaine mise à jour
        await asyncio.sleep(time_step)

def start_plot(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    running = True

    global etat_courant, Panne
    Panne = False
    etat_courant = "etat_Consom"

    # Démarrer la mise à jour des données
    asyncio.ensure_future(update_data())

def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t, etat_courant, Mg, PFC
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    etat_courant = "etat_Consom"  # Réinitialiser l'état
    Mg = 0.045  # Réinitialiser Mg
    PFC = 3500  # Réinitialiser PFC
    with fig1.batch_update():
        fig1.data = []  # Effacer les données existantes
        fig1.add_scatter(x=timedata, y=Mgdata, line=dict(color='green'))
    with fig2.batch_update():
        scatter2.update(x=timedata, y=PFCdata)

def serie_plot(button):
    """Fonction pour passer à l'état de série."""
    global etat_courant
    etat_courant = "etat_Serie"

def panne_plot(button):
    """Fonction pour passer à l'état de panne."""
    global etat_courant, Panne
    Panne = True
    etat_courant = "etat_Panne"

def poche_plot(button):
    """Fonction pour passer à l'état de poche."""
    global etat_courant, Poche
    Poche = True
    etat_courant = "etat_Consom"

def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start")
    reset_button = widgets.Button(description="Reset")
    stop_button = widgets.Button(description="Stop")
    poche_button = widgets.Button(description="Poche")
    serie_button = widgets.Button(description="Série")
    panne_button = widgets.Button(description="Panne")

    start_button.on_click(start_plot)
    reset_button.on_click(reset_plot)
    stop_button.on_click(stop_plot)
    poche_button.on_click(poche_plot)
    serie_button.on_click(serie_plot)
    panne_button.on_click(panne_plot)

    display(widgets.HBox([start_button, reset_button, stop_button, poche_button, serie_button, panne_button]), fig1, fig2)

def main(initial_time_step=1, initial_t=0, Mgmin=0.05, Mgmax=0.1, Mglimite=0.02):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t, Mgmin, Mgmax, Mglimite)
    global fig1, scatter1, fig2, scatter2
    fig1, fig2 = create_figures()
    scatter1 = fig1.data[0]
    scatter2 = fig2.data[0]
    create_buttons()

# Appel de la fonction principale pour démarrer l'application
main()


FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'a9e7bee8-518c-4349-b651-e6e31f29dde3'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Mg'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Mg'}}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': 'ada4b531-9127-49e1-b6a3-90c84e65eb11'}],
    'layout': {'template': '...',
               'title': {'text': 'Consommation de Fonte'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Fonte'}}}
})